In [1]:
from pyspark.sql import SparkSession

spark: SparkSession = SparkSession.builder \
                        .appName('Test Silver') \
                        .config("spark.hadoop.fs.defaultFS", "file:///") \
                        .config("spark.driver.memory", "4g") \
                        .config("spark.sql.shuffle.partitions", "400") \
                        .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.3.0") \
                        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
                        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
                        .getOrCreate()

25/08/21 17:53:50 WARN Utils: Your hostname, DESKTOP-9VM3SA1 resolves to a loopback address: 127.0.1.1; using 172.28.82.250 instead (on interface eth0)
25/08/21 17:53:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/dottier/big_data/venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/dottier/.ivy2/cache
The jars for the packages stored in: /home/dottier/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-abf92ae8-7f22-47c8-9ac4-ec161aaa1048;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 140ms :: artifacts dl 7ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.0 from central in [default]
	io.delta#delta-storage;3.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   

In [2]:
print(spark.version)  # Spark version
print(spark.sparkContext.getConf().get("spark.scala.version")) # Scala version


3.5.6
None


In [3]:
print(spark._jsc.hadoopConfiguration().get("fs.defaultFS"))

file:///


In [2]:
import re
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

def camel_to_snake(name: str) -> str:
    if name is None:
        return None
    return re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower()

In [3]:
import re
import pandas as pd
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import StringType

# 1. The core logic remains the same, but it now operates on a Pandas Series.
def camel_to_snake_vectorized(series: pd.Series) -> pd.Series:
    # The .str accessor in Pandas applies the regex to the whole series at once.
    return series.str.replace(r'(?<!^)(?=[A-Z])', '_', regex=True).str.lower()

# 2. Register it as a Pandas UDF.
snake_case_pandas_udf = pandas_udf(camel_to_snake_vectorized, returnType=StringType())

In [ ]:
# from pyspark.sql import DataFrame
# from pyspark.sql.functions import col, explode_outer
# from pyspark.sql.types import StructType, ArrayType

# # Recursively flattens a PySpark DataFrame
# def flatten_df(df: DataFrame, parent_prefix: str = "") -> DataFrame:
#     flat_cols = []
#     explode_cols = []
    
#     for field in df.schema.fields:
#         field_name = field.name
#         col_name = f"{parent_prefix}{field_name}" if parent_prefix else field_name
#         alias_name = camel_to_snake(col_name.replace(".", "_"))
#         data_type = field.dataType

#         if isinstance(data_type, StructType):
#             # Recurse into struct
#             for subfield in data_type.fields:
#                 sub_col_name = f"{col_name}.{subfield.name}"
#                 sub_alias = camel_to_snake(sub_col_name.replace(".", "_"))
#                 flat_cols.append(col(sub_col_name).alias(sub_alias))
        
#         elif isinstance(data_type, ArrayType) and isinstance(data_type.elementType, StructType):
#             # Explode arrays of structs
#             explode_cols.append((col_name, field_name))
        
#         else:
#             # Primitive column
#             flat_cols.append(col(col_name).alias(alias_name))
    
#     return df.select(flat_cols)


In [4]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, explode_outer
from pyspark.sql.types import StructType, StringType

def flatten_df(df: DataFrame, cols_to_snake_case_values: set = None) -> DataFrame:
    if cols_to_snake_case_values is None:
        cols_to_snake_case_values = set()

    flat_cols = []
    
    def get_flat_cols(schema: StructType, prefix: str = ""):
        for field in schema.fields:
            full_col_name = f"{prefix}{field.name}"
            
            # Recurse into nested structs
            if isinstance(field.dataType, StructType):
                get_flat_cols(field.dataType, prefix=f"{full_col_name}.")
            else:
                # Create the final, clean snake_case alias
                alias_name = camel_to_snake(full_col_name.replace(".", "_"))
                
                # Check if this column's values should be cleaned
                if alias_name in cols_to_snake_case_values and isinstance(field.dataType, StringType):
                    flat_cols.append(
                        snake_case_pandas_udf(col(full_col_name)).alias(alias_name)
                    )
                else:
                    flat_cols.append(
                        col(full_col_name).alias(alias_name)
                    )

    # Start the recursion on the top-level schema
    get_flat_cols(df.schema)
    
    # Return the DataFrame with the flattened and aliased columns
    return df.select(flat_cols)


In [5]:
import os

BRONZE_PATH = "/home/dottier/big_data/bronze"
STAGE_PATH = os.path.join(BRONZE_PATH, "stage_data")
MATCH_PATH = os.path.join(BRONZE_PATH, "match_data")

SILVER_PATH = "/home/dottier/big_data/silver"

In [6]:
from delta.tables import DeltaTable

def write_to_silver(df: DataFrame, table_name: str, is_league_partition: bool = False):
    table_path = f"{SILVER_PATH}/{table_name}"

    if not DeltaTable.isDeltaTable(spark, table_path):
        empty_df = spark.createDataFrame([], df.schema)
        initial_writer = empty_df.write.format("delta")
        
        if is_league_partition:
            if "league" not in df.columns:
                raise ValueError("DataFrame does not contain a 'league' column for partitioning")
            initial_writer = initial_writer.partitionBy("league")

        initial_writer.save(table_path)
        print("Empty table created.")

    writer = df.write.mode("append").format("delta")
    if is_league_partition:
        writer = writer.partitionBy("league")
    
    writer.save(table_path)

In [43]:
from delta.tables import DeltaTable

def read_from_silver(table_name: str):
    table_path = f"{SILVER_PATH}/{table_name}"
    df = spark.read.format("delta").load(table_path)
    return df

In [13]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import asc, desc, col

stage_schemas = StructType([
    StructField("regionId", IntegerType(), False),
    StructField("regionName", StringType(), True),
    StructField("tournamentId", IntegerType(), False),
    StructField("tournamentName", StringType(), True),
    StructField("seasonId", IntegerType(), False),
    StructField("seasonName", StringType(), False),
    StructField("stageName", StringType(), False),
])

dim_stages_df = spark.read \
    .schema(stage_schemas) \
    .option("multiLine", "true") \
    .json(STAGE_PATH)

dim_stages_df = flatten_df(dim_stages_df)

In [14]:
dim_stages_df.printSchema()
print(dim_stages_df.count())
dim_stages_df.show(truncate=False)

root
 |-- region_id: integer (nullable = true)
 |-- region_name: string (nullable = true)
 |-- tournament_id: integer (nullable = true)
 |-- tournament_name: string (nullable = true)
 |-- season_id: integer (nullable = true)
 |-- season_name: string (nullable = true)
 |-- stage_name: string (nullable = true)
 |-- league: string (nullable = true)
 |-- season: string (nullable = true)
 |-- stage_id: integer (nullable = true)

140
+---------+-----------+-------------+----------------+---------+-----------+----------------+-----------------------+---------+--------+
|region_id|region_name|tournament_id|tournament_name |season_id|season_name|stage_name      |league                 |season   |stage_id|
+---------+-----------+-------------+----------------+---------+-----------+----------------+-----------------------+---------+--------+
|252      |England    |2            |Premier League  |8228     |2020/2021  |Premier League  |england-premier-league |2020-2021|18685   |
|250      |Europe   

In [15]:
write_to_silver(dim_stages_df, "dim_stages")

Empty table created.


In [9]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col, expr

team_schema = StructType([
    StructField("homeTeamId", IntegerType(), False),
    StructField("homeTeamName", StringType(), False),
    StructField("homeTeamCountryName", StringType(), False),
    StructField("awayTeamId", IntegerType(), False),
    StructField("awayTeamName", StringType(), False),
    StructField("awayTeamCountryName", StringType(), False),
])

dim_teams_df = spark.read \
    .schema(team_schema) \
    .option("multiLine", True) \
    .option("pathGlobFilter", "*match_preview.json") \
    .json(MATCH_PATH)

# or select home_df and away_df then join
dim_teams_df = dim_teams_df.select(
    expr("stack(2, homeTeamId, homeTeamName, homeTeamCountryName, awayTeamId, awayTeamName, awayTeamCountryName) as (team_id, team_name, country_name)"),
).distinct()

In [30]:
dim_teams_df.printSchema()
print(dim_teams_df.count())
dim_teams_df.show(10)

root
 |-- team_id: integer (nullable = true)
 |-- team_name: string (nullable = true)
 |-- country_name: string (nullable = true)

186
+-------+-------------------+------------+
|team_id|          team_name|country_name|
+-------+-------------------+------------+
|    282|            FC Koln|     Germany|
|   3429|         Ingolstadt|     Germany|
|     44|  Borussia Dortmund|     Germany|
|    140|               Genk|     Belgium|
|    595|            Maribor|    Slovenia|
|     45|Eintracht Frankfurt|     Germany|
|    133|           Besiktas|      Turkey|
|    847|        CSKA Moscow|      Russia|
|   1030|     Legia Warszawa|      Poland|
|     68|      Real Sociedad|       Spain|
+-------+-------------------+------------+
only showing top 10 rows



In [10]:
(
    dim_teams_df
    .write
    .mode("append")
    .format("delta")
    .save(f"{SILVER_PATH}/dim_teams")
)

In [13]:
dim_teams_df.filter(col("team_name") == col("country_name")).show()

+-------+-----------+------------+
|team_id|  team_name|country_name|
+-------+-----------+------------+
|   1293|       Iran|        Iran|
|    345|    England|     England|
|    409|     Brazil|      Brazil|
|    336|    Germany|     Germany|
|    423|Switzerland| Switzerland|
|    340|   Portugal|    Portugal|
|    341|     France|      France|
|    346|  Argentina|   Argentina|
|    339|    Belgium|     Belgium|
|    959|    Tunisia|     Tunisia|
|   2694|     Panama|      Panama|
|    338|      Spain|       Spain|
|    970| Costa Rica|  Costa Rica|
|    863|   Cameroon|    Cameroon|
|    771|     Serbia|      Serbia|
|   1159|South Korea| South Korea|
|    966|    Algeria|     Algeria|
|    337|    Croatia|     Croatia|
|    335|Netherlands| Netherlands|
|    328|  Australia|   Australia|
+-------+-----------+------------+
only showing top 20 rows



In [6]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType
from pyspark.sql.functions import desc, col, regexp_extract

match_preview_schemas = StructType([
    StructField(("homeTeamId"), IntegerType(), True),
    StructField(("awayTeamId"), IntegerType(), True),
    StructField(("startTimeUtc"), TimestampType(), True)
])

match_preview_df = spark.read \
    .schema(match_preview_schemas) \
    .option("multiLine", "true") \
    .option("pathGlobFilter", "*match_preview.json") \
    .json(MATCH_PATH)

match_preview_df = flatten_df(match_preview_df.drop("league", "season"))

In [8]:
match_preview_df.printSchema()
# print(match_preview_df.count())
match_preview_df.orderBy(
    # col("stage_id").asc(),
    col("match_id").desc()
).show()

root
 |-- home_team_id: integer (nullable = true)
 |-- away_team_id: integer (nullable = true)
 |-- start_time_utc: timestamp (nullable = true)
 |-- stage_id: integer (nullable = true)
 |-- match_id: integer (nullable = true)

+------------+------------+-------------------+--------+--------+
|home_team_id|away_team_id|     start_time_utc|stage_id|match_id|
+------------+------------+-------------------+--------+--------+
|          42|          44|2026-05-16 13:30:00|   24478| 1910902|
|         796|        1730|2026-05-16 13:30:00|   24478| 1910901|
|         283|          33|2026-05-16 13:30:00|   24478| 1910900|
|        4852|         219|2026-05-16 13:30:00|   24478| 1910899|
|          50|        7614|2026-05-16 13:30:00|   24478| 1910898|
|          45|          41|2026-05-16 13:30:00|   24478| 1910897|
|         134|        1211|2026-05-16 13:30:00|   24478| 1910896|
|          37|         282|2026-05-16 13:30:00|   24478| 1910895|
|          36|          38|2026-05-16 13:30:00|

In [12]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import desc, col, regexp_extract, split, regexp_extract

match_data_schemas = StructType([
    StructField("matchId", IntegerType(), True),
    StructField("matchCentreData", StructType([
        StructField(("score"), StringType(), False),
        StructField(("htScore"), StringType(), False),
        StructField(("ftScore"), StringType(), False),
        StructField(("etScore"), StringType(), False),
        StructField(("pkScore"), StringType(), False),
    ]))
])

number_pattern = r"(\d+)"

match_data_df = spark.read \
    .schema(match_data_schemas) \
    .option("multiLine", "true") \
    .option("pathGlobFilter", "*match_data.json") \
    .json(MATCH_PATH)

match_data_df = match_data_df.select(
    col("matchId").alias("match_id"),
    regexp_extract(split(col("matchCentreData.score"), ":").getItem(0), number_pattern, 1).cast("integer").alias("home_score"),
    regexp_extract(split(col("matchCentreData.score"), ":").getItem(1), number_pattern, 1).cast("integer").alias("away_score"),
    regexp_extract(split(col("matchCentreData.htScore"), ":").getItem(0), number_pattern, 1).cast("integer").alias("home_ht_score"),
    regexp_extract(split(col("matchCentreData.htScore"), ":").getItem(1), number_pattern, 1).cast("integer").alias("away_ht_score"),
    regexp_extract(split(col("matchCentreData.ftScore"), ":").getItem(0), number_pattern, 1).cast("integer").alias("home_ft_score"),
    regexp_extract(split(col("matchCentreData.ftScore"), ":").getItem(1), number_pattern, 1).cast("integer").alias("away_ft_score"),
    regexp_extract(split(col("matchCentreData.etScore"), ":").getItem(0), number_pattern, 1).cast("integer").alias("home_et_score"),
    regexp_extract(split(col("matchCentreData.etScore"), ":").getItem(1), number_pattern, 1).cast("integer").alias("away_et_score"),
    regexp_extract(split(col("matchCentreData.pkScore"), ":").getItem(0), number_pattern, 1).cast("integer").alias("home_pk_score"),
    regexp_extract(split(col("matchCentreData.pkScore"), ":").getItem(1), number_pattern, 1).cast("integer").alias("away_pk_score"),
    col("stage_id")
)

In [13]:
match_data_df.printSchema()
print(match_data_df.count())
match_data_df.show()

root
 |-- match_id: integer (nullable = true)
 |-- home_score: integer (nullable = true)
 |-- away_score: integer (nullable = true)
 |-- home_ht_score: integer (nullable = true)
 |-- away_ht_score: integer (nullable = true)
 |-- home_ft_score: integer (nullable = true)
 |-- away_ft_score: integer (nullable = true)
 |-- home_et_score: integer (nullable = true)
 |-- away_et_score: integer (nullable = true)
 |-- home_pk_score: integer (nullable = true)
 |-- away_pk_score: integer (nullable = true)
 |-- league: string (nullable = true)
 |-- season: string (nullable = true)
 |-- stage_id: integer (nullable = true)



7697
+--------+----------+----------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+--------------------+---------+--------+
|match_id|home_score|away_score|home_ht_score|away_ht_score|home_ft_score|away_ft_score|home_et_score|away_et_score|home_pk_score|away_pk_score|              league|   season|stage_id|
+--------+----------+----------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+--------------------+---------+--------+
| 1789439|         2|         1|            1|            1|            2|            1|            2|            1|            3|            2|europe-champions-...|2023-2024|   22686|
| 1809759|         1|         1|            0|            1|            1|            1|            1|            1|            3|            4|europe-champions-...|2023-2024|   22686|
| 1891173|         1|         0|            1|            0|          

In [19]:
match_data_df.unpersist()

DataFrame[match_id: int, home_score: int, away_score: int, home_ht_score: int, away_ht_score: int, home_ft_score: int, away_ft_score: int, home_et_score: int, away_et_score: int, home_pk_score: int, away_pk_score: int, stage_id: int]

In [7]:
match_data_df.filter(col("away_pk_score").isNotNull()).show()

+--------+----------+----------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+--------------------+---------+--------+
|match_id|home_score|away_score|home_ht_score|away_ht_score|home_ft_score|away_ft_score|home_et_score|away_et_score|home_pk_score|away_pk_score|              league|   season|stage_id|
+--------+----------+----------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+--------------------+---------+--------+
| 1789439|         2|         1|            1|            1|            2|            1|            2|            1|            3|            2|europe-champions-...|2023-2024|   22686|
| 1809759|         1|         1|            0|            1|            1|            1|            1|            1|            3|            4|europe-champions-...|2023-2024|   22686|
| 1891173|         1|         0|            1|            0|            1| 

In [28]:
match_data_df.filter(col("home_pk_score").isNotNull()).show()

+--------+----------+----------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+--------------------+---------+--------+
|match_id|home_score|away_score|home_ht_score|away_ht_score|home_ft_score|away_ft_score|home_et_score|away_et_score|home_pk_score|away_pk_score|              league|   season|stage_id|
+--------+----------+----------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+--------------------+---------+--------+
| 1789439|         2|         1|            1|            1|            2|            1|            2|            1|            3|            2|europe-champions-...|2023-2024|   22686|
| 1809759|         1|         1|            0|            1|            1|            1|            1|            1|            3|            4|europe-champions-...|2023-2024|   22686|
| 1891173|         1|         0|            1|            0|            1| 

In [13]:
fct_match_summary_df = match_preview_df.join(
    other=match_data_df,
    on=["match_id", "stage_id"],
    how="left_outer")

In [22]:
print(fct_match_summary_df.count())
fct_match_summary_df.printSchema()

fct_match_summary_df.orderBy(
    col("match_id")
).show(10)

8902
root
 |-- match_id: integer (nullable = true)
 |-- stage_id: integer (nullable = true)
 |-- home_team_id: integer (nullable = true)
 |-- away_team_id: integer (nullable = true)
 |-- start_time_utc: timestamp (nullable = true)
 |-- home_score: integer (nullable = true)
 |-- away_score: integer (nullable = true)
 |-- home_ht_score: integer (nullable = true)
 |-- away_ht_score: integer (nullable = true)
 |-- home_ft_score: integer (nullable = true)
 |-- away_ft_score: integer (nullable = true)
 |-- home_et_score: integer (nullable = true)
 |-- away_et_score: integer (nullable = true)
 |-- home_pk_score: integer (nullable = true)
 |-- away_pk_score: integer (nullable = true)



+--------+--------+------------+------------+-------------------+----------+----------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
|match_id|stage_id|home_team_id|away_team_id|     start_time_utc|home_score|away_score|home_ht_score|away_ht_score|home_ft_score|away_ft_score|home_et_score|away_et_score|home_pk_score|away_pk_score|
+--------+--------+------------+------------+-------------------+----------+----------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
|  789612|    7558|         338|         418|2014-06-18 19:00:00|         0|         2|            0|            2|            0|            2|         NULL|         NULL|         NULL|         NULL|
|  789613|    7557|         863|         409|2014-06-23 20:00:00|         1|         4|            1|            2|            1|            4|         NULL|         NULL|         NULL|         NULL|


In [15]:
(
    fct_match_summary_df
    .write
    .mode("append")
    .format("delta")
    .save(f"{SILVER_PATH}/fct_match_summary")
)

In [16]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col, expr

team_stage_schema = StructType([
    StructField("homeTeamId", IntegerType(), False),
    StructField("awayTeamId", IntegerType(), False),
])

fct_team_stage_participation_df = spark.read \
    .schema(team_stage_schema) \
    .option("multiLine", True) \
    .option("pathGlobFilter", "*match_preview.json") \
    .json(MATCH_PATH)

# or select home_df and away_df then join
fct_team_stage_participation_df = fct_team_stage_participation_df.select(
    expr("stack(2, homeTeamId, awayTeamId) as (team_id)"),
    col("stage_id")
).distinct()

In [33]:
print(fct_team_stage_participation_df.count())
fct_team_stage_participation_df.printSchema()
fct_team_stage_participation_df.show()

1086
root
 |-- team_id: integer (nullable = true)
 |-- stage_id: integer (nullable = true)

+-------+--------+
|team_id|stage_id|
+-------+--------+
|    493|   12857|
|   7614|   18979|
|    328|   18652|
|    189|   24533|
|   4852|   23471|
|     24|   19793|
|    768|    7562|
|    124|   16652|
|     30|   18065|
|    130|   20961|
|     41|   12559|
|    299|   20265|
|     84|   15485|
|    607|   24083|
|    970|    7566|
|   7614|   13872|
|     44|   20965|
|    288|   22492|
|    304|   13185|
|    421|   18650|
+-------+--------+
only showing top 20 rows



In [17]:
(
    fct_team_stage_participation_df
    .write
    .mode("append")
    .format("delta")
    .save(f"{SILVER_PATH}/fct_team_stage_partcipation")
)

In [18]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ArrayType
from pyspark.sql.functions import col, expr, explode

player_schema = StructType([
    StructField("matchCentreData", StructType([
        StructField("home", StructType([
            StructField("players", ArrayType(
                StructType([
                    StructField("playerId", IntegerType(), False),
                    StructField("name", StringType(), False)
                ])
            ))
        ])),
        StructField("away", StructType([
            StructField("players", ArrayType(
                StructType([
                    StructField("playerId", IntegerType(), False),
                    StructField("name", StringType(), False)
                ])
            ))
        ]))
    ]))
])

dim_players_df = spark.read \
    .schema(player_schema) \
    .option("multiLine", True) \
    .option("pathGlobFilter", "*match_data.json") \
    .json(MATCH_PATH)

home_df = dim_players_df.select(explode("matchCentreData.home.players").alias("player"))
home_df = home_df.select(col("player.playerId"), col("player.name"))

away_df = dim_players_df.select(explode("matchCentreData.away.players").alias("player"))
away_df = away_df.select(col("player.playerId"), col("player.name"))

dim_players_df = home_df.union(away_df)

dim_players_df = dim_players_df.select(
    col("playerId").alias("player_id"),
    col("name").alias("player_name")
).dropDuplicates(["player_id"])

In [36]:
print(dim_players_df.count())
dim_players_df.printSchema()
dim_players_df.show()

7856
root
 |-- player_id: integer (nullable = true)
 |-- player_name: string (nullable = true)

+---------+-------------------+
|player_id|        player_name|
+---------+-------------------+
|     5803|  Olivier Deschacht|
|    10623|Michael Krohn-Dehli|
|    15846|      Yasuhito Endo|
|    29814|         Max Gradel|
|    34239|   Sofiane Féghouli|
|    35689|    Gabriel Mercado|
|    70097|     Diego Contento|
|    83250|    Aymen Abdennour|
|    96853|        Tom Trybull|
|   128589|      Ejike Uzoenyi|
|   202251|    Emanuel Mammana|
|   260819|         Akram Afif|
|   275698|       Adam Marusic|
|   278125|     Grischa Prömel|
|   279177|     Marcus Edwards|
|   294744|        Luiz Felipe|
|   300779|          Nacho Gil|
|   322881|     Moritz Nicolas|
|   323250|      Rubén Sobrino|
|   326371|           Zé Gomes|
+---------+-------------------+
only showing top 20 rows



In [20]:
write_to_silver(dim_players_df, "dim_players")

25/08/14 18:47:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/08/14 18:47:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/08/14 18:47:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/08/14 18:47:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/08/14 18:47:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/08/14 18:47:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
25/08/14 18:47:09 WARN MemoryManager: Total allocation exceeds 95.

In [37]:
view = dim_players_df.filter(col("player_name").contains("Mus"))
view.show()

+---------+-------------------+
|player_id|        player_name|
+---------+-------------------+
|   126278|         Juan Musso|
|    61014|    Mateo Musacchio|
|    93577|         Ahmed Musa|
|   350076|        Musa Barrow|
|   338493|    Florent Muslija|
|   381792|         Al Musrati|
|   395252|      Jamal Musiala|
|    30966|   Fernando Muslera|
|   447649|      Musaab Khidir|
|   400357|        Yunus Musah|
|    83167|Farouk Ben Mustapha|
|   421777|      Alen Mustafic|
|    34022|   Mustafa Pektemek|
|   424589|    Marlon Mustapha|
|   297353|         Mario Musa|
|   367815|        Leon Musaev|
|    80921|   Shkodran Mustafi|
|   303172|     Charly Musonda|
|   404198|         Petar Musa|
+---------+-------------------+



In [9]:
def extract_side(df: DataFrame, side):
    return df.select(
        explode(f"matchCentreData.{side}.players").alias("player"),
        col(f"matchCentreData.{side}.teamId").alias("team_id"),
        col("match_id"),
        col("stage_id")
    )
    # .withColumnRenamed(
    #     "player.playerId", "player_id"
    # ).withColumnRenamed(
    #     "player.shirtNo", "player_id"
    # ).withColumnRenamed(
    #     "player.playerId", "player_id"
    # )

In [14]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ArrayType, BooleanType
from pyspark.sql.functions import col, expr, explode

player_match_schema = StructType([
    StructField("matchCentreData", StructType([
        StructField("home", StructType([
            StructField("teamId", IntegerType(), False),
            StructField("players", ArrayType(
                StructType([
                    StructField("playerId", IntegerType(), False),
                    StructField("shirtNo", IntegerType(), False),
                    StructField("isFirstEleven", BooleanType(), False)
                ])
            ), True)
        ]), True),
        StructField("away", StructType([
            StructField("teamId", IntegerType(), False),
            StructField("players", ArrayType(
                StructType([
                    StructField("playerId", IntegerType(), False),
                    StructField("shirtNo", IntegerType(), False),
                    StructField("isFirstEleven", BooleanType(), False)
                ])
            ), True)
        ]), True)
    ]))
])

fct_player_match_participation_df = spark.read \
    .schema(player_match_schema) \
    .option("multiLine", True) \
    .option("pathGlobFilter", "*match_data.json") \
    .json(MATCH_PATH)

home_df = extract_side(fct_player_match_participation_df, "home")
away_df = extract_side(fct_player_match_participation_df, "away")
fct_player_match_participation_df = home_df.union(away_df)

fct_player_match_participation_df = flatten_df(
    fct_player_match_participation_df.select(
        col("match_id"),
        col("team_id"),
        col("player.*")
    )
)

fct_player_match_participation_df.fillna(False, ["is_first_eleven"])

DataFrame[match_id: int, team_id: int, player_id: int, shirt_no: int, is_first_eleven: boolean]

In [15]:
print(fct_player_match_participation_df.count())
fct_player_match_participation_df.printSchema()
fct_player_match_participation_df.show()

295816
root
 |-- match_id: integer (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- player_id: integer (nullable = true)
 |-- shirt_no: integer (nullable = true)
 |-- is_first_eleven: boolean (nullable = true)

+--------+-------+---------+--------+---------------+
|match_id|team_id|player_id|shirt_no|is_first_eleven|
+--------+-------+---------+--------+---------------+
| 1789439|     63|    76662|      13|           true|
| 1789439|     63|    25191|      20|           true|
| 1789439|     63|    77564|      15|           true|
| 1789439|     63|   344156|      22|           true|
| 1789439|     63|   303728|      16|           true|
| 1789439|     63|   381594|      12|           true|
| 1789439|     63|   125547|       5|           true|
| 1789439|     63|   254582|      14|           true|
| 1789439|     63|    80764|       6|           true|
| 1789439|     63|    80241|       7|           true|
| 1789439|     63|    91213|      19|           true|
| 1789439|     63| 

In [16]:
write_to_silver(fct_player_match_participation_df, "fct_player_match_participation")

Empty table created.


In [ ]:
view_2 = fct_player_match_participation_df.filter(col("player_id") == 13754)
view_2.show()

+---------+-------+--------+
|player_id|team_id|stage_id|
+---------+-------+--------+
|    13754|     37|   17682|
|    13754|     37|   22686|
|    13754|     37|   21026|
|    13754|     37|   16427|
|    13754|    336|    7563|
|    13754|     37|   14361|
|    13754|     37|   19130|
|    13754|     37|   18762|
|    13754|     37|   20265|
|    13754|     37|   13185|
|    13754|     37|   15243|
|    13754|    336|    7567|
|    13754|     37|   24083|
|    13754|    336|    7566|
|    13754|    336|    7569|
|    13754|     37|   22128|
|    13754|    336|   12756|
|    13754|     37|   16655|
|    13754|     37|   16651|
|    13754|     37|   19862|
+---------+-------+--------+
only showing top 20 rows



In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ArrayType
from pyspark.sql.functions import col, expr, explode

match_lineup_schema = StructType([
    StructField("matchCentreData", StructType([
        StructField("home", StructType([
            StructField("teamId", IntegerType(), False),
            StructField("players", ArrayType(
                StructType([
                    StructField("playerId", IntegerType(), False)
                ])
            ), True)
        ]), True),
        StructField("away", StructType([
            StructField("teamId", IntegerType(), False),
            StructField("players", ArrayType(
                StructType([
                    StructField("playerId", IntegerType(), False)
                ])
            ), True)
        ]), True)
    ]))
])

fct_player_match_participation_df = spark.read \
    .schema(player_match_schema) \
    .option("multiLine", True) \
    .option("pathGlobFilter", "*match_data.json") \
    .json(MATCH_PATH)

home_df = extract_side(fct_player_match_participation_df, "home")
away_df = extract_side(fct_player_match_participation_df, "away")
fct_player_match_participation_df = home_df.union(away_df)

fct_player_match_participation_df = fct_player_match_participation_df.select(
    col("player_id"),
    col("team_id"),
    col("match_id")
).dropDuplicates()

In [8]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, ArrayType, BooleanType
from pyspark.sql.functions import col, explode_outer, asc, desc, row_number, monotonically_increasing_id, explode
from pyspark.sql.window import Window

match_event_schema = StructType([
    StructField("matchCentreData", StructType([
        StructField("events", ArrayType(
            StructType([
                StructField("eventId", IntegerType(), False),
                StructField("minute", IntegerType(), False),
                StructField("second", IntegerType(), False),
                StructField("teamId", IntegerType(), False),
                StructField("playerId", IntegerType(), False),
                StructField("x", FloatType(), False),
                StructField("y", FloatType(), False),
                StructField("period", StructType([
                    StructField("value", IntegerType(), False),
                    StructField("displayName", StringType(), False),
                ])),
                StructField("type", StructType([
                    StructField("value", IntegerType(), False),
                    StructField("displayName", StringType(), False),
                ])),
                StructField("outcomeType", StructType([
                    StructField("value", IntegerType(), False),
                    StructField("displayName", StringType(), False),
                ])),
                StructField("satisfiedEventsTypes", ArrayType(IntegerType()), True)
            ])
        ), False)
    ]))
])

fct_match_events_df = spark.read \
    .schema(match_event_schema) \
    .option("multiLine", True) \
    .option("pathGlobFilter", "*match_data.json") \
    .json(MATCH_PATH)

# # repartition before exploding for low shuffle cost later
# num_partitions = 200
# fct_match_events_df = fct_match_events_df.repartition(num_partitions, "match_id")

# exploding column "event"
fct_match_events_df = fct_match_events_df.withColumn("event", explode_outer(col("matchCentreData.events"))).drop("matchCentreData")

# Since event_id is unique for each team per match
# We create an unique _match_id to preserve the original event order of every match
fct_match_events_df = fct_match_events_df.withColumn(
    "surrogate_event_id", monotonically_increasing_id()
)
w = Window.partitionBy("match_id").orderBy("surrogate_event_id")
fct_match_events_df = fct_match_events_df.withColumn("_event_id", row_number().over(w))

base_match_events_df = (
    flatten_df(
        fct_match_events_df.select(
            col("surrogate_event_id"),
            col("match_id"),
            col("_event_id"),
            "event.*",
            col("league"),
            col("season"),
            col("stage_id")
        )
    )
)

exploded_match_events_df = base_match_events_df.select(
    col("surrogate_event_id"),
    explode(col("satisfied_events_types")).alias("event_type_id")
)

# fct_match_events_df = fct_match_events_df.orderBy(
#     asc("match_id"),
#     asc("_event_id")
# )

In [7]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, ArrayType, BooleanType
from pyspark.sql.functions import col, explode_outer, asc, desc, row_number, monotonically_increasing_id, explode, transform, filter, regexp_replace, lower, when
from pyspark.sql.window import Window

match_event_schema = StructType([
    StructField("matchCentreData", StructType([
        StructField("events", ArrayType(
            StructType([
                StructField("eventId", IntegerType(), False),
                StructField("minute", IntegerType(), False),
                StructField("second", IntegerType(), False),
                StructField("teamId", IntegerType(), False),
                StructField("playerId", IntegerType(), False),
                StructField("x", FloatType(), False),
                StructField("y", FloatType(), False),
                StructField("endX", FloatType(), True),
                StructField("endY", FloatType(), True),
                StructField("period", StructType([
                    StructField("value", IntegerType(), False),
                    StructField("displayName", StringType(), False),
                ])),
                StructField("type", StructType([
                    StructField("displayName", StringType(), False),
                ])),
                StructField("outcomeType", StructType([
                    StructField("value", IntegerType(), False),
                ])),
                StructField("qualifiers", ArrayType(
                    StructType([
                        StructField("type", StructType([
                            StructField("value", IntegerType(), False),
                            StructField("displayName", StringType(), False),
                        ])),
                        StructField("value", StringType(), True)
                    ])
                ), True),
                StructField("satisfiedEventsTypes", ArrayType(IntegerType()), True)
            ])
        ), False)
    ]))
])

fct_match_events_df = spark.read \
    .schema(match_event_schema) \
    .option("multiLine", True) \
    .option("pathGlobFilter", "*match_data.json") \
    .json(MATCH_PATH)

# exploding column "event"
fct_match_events_df = fct_match_events_df.withColumn("event", explode_outer(col("matchCentreData.events"))).drop("matchCentreData")

# Explode and filter qualifiers to get qualifiers mapping
# Example: "StandingSave" -> 178
qualifier_mapping_df = (
    fct_match_events_df
    .select("event.qualifiers")
    .filter(col("qualifiers").isNotNull())
    
    # Keep only qualifiers without values
    .select(explode("qualifiers").alias("q"))
    .filter(col("q.value").isNull())
    
    # remove dups
    .select(
        col("q.type.value").alias("qualifier_id"),
        col("q.type.displayName").alias("qualifier_name_raw")
    )
    .distinct()
    
    # to snake_case
    .withColumn(
        "qualifier_name",
        lower(regexp_replace(col("qualifier_name_raw"), r"([a-z])([A-Z])", r"$1_$2"))
    )
    .select("qualifier_name", "qualifier_id")
)


fct_match_events_df = fct_match_events_df.withColumn(
    "qualifiers_display_names",
    transform(
        # Filter the array to keep only flags
        filter(
            col("event.qualifiers"),
            lambda q: q.value.isNull()
        ),
        # Transform the filtered array to get the names
        lambda q: lower(
            regexp_replace(q.type.displayName, r"([a-z])([A-Z])", r"$1_$2")
        )
    )
).withColumn(
    "qualifiers_values",
    transform(
        filter(
            col("event.qualifiers"),
            lambda q: q.value.isNull()
        ),
        lambda q: (q.type.value)
    )
).withColumn(
    "event",
    col("event").dropFields("qualifiers")
)

# Since event_id is unique for each team per match
# We create an unique _event_id to preserve the original event order of every match
fct_match_events_df = fct_match_events_df.withColumn(
    "surrogate_event_id", monotonically_increasing_id()
)
w = Window.partitionBy("match_id").orderBy("surrogate_event_id")
fct_match_events_df = fct_match_events_df.withColumn("_event_id", row_number().over(w))

df_to_flatten = fct_match_events_df.select(
    col("surrogate_event_id"),
    col("match_id"),
    col("_event_id"),
    "event.*",
    col("qualifiers_display_names"),
    col("qualifiers_values"),
    col("league"),
    col("season"),
    col("stage_id")
)

categorical_cols_to_clean = {
    "period_display_name",
    "type_display_name"
}

base_match_events_df = flatten_df(
    df_to_flatten, categorical_cols_to_clean
)

base_match_events_df = flatten_df(
    df_to_flatten, categorical_cols_to_clean
).withColumn(
    "is_successful", when(col("outcome_type_value") == 1, True).otherwise(False)
).drop(
    "outcome_type_value", "surrogate_event_id"
)


# exploded_match_events_df = base_match_events_df.select(
#     col("surrogate_event_id"),
#     explode(col("satisfied_events_types")).alias("event_type_id")
# )

# fct_match_events_df = fct_match_events_df.orderBy(
#     asc("match_id"),
#     asc("_event_id")
# )

In [8]:
write_to_silver(qualifier_mapping_df, "qualifier_mapping")

Empty table created.


In [8]:
# Hardcoded original mapping
original_mapping = {
    "shotSixYardBox": 0,
    "shotPenaltyArea": 1,
    "shotOboxTotal": 2,
    "shotOpenPlay": 3,
    "shotCounter": 4,
    "shotSetPiece": 5,
    "shotDirectCorner": 6,
    "shotOffTarget": 7,
    "shotOnPost": 8,
    "shotOnTarget": 9,
    "shotsTotal": 10,
    "shotBlocked": 11,
    "shotRightFoot": 12,
    "shotLeftFoot": 13,
    "shotHead": 14,
    "shotObp": 15,
    "goalSixYardBox": 16,
    "goalPenaltyArea": 17,
    "goalObox": 18,
    "goalOpenPlay": 19,
    "goalCounter": 20,
    "goalSetPiece": 21,
    "penaltyScored": 22,
    "goalOwn": 23,
    "goalNormal": 24,
    "goalRightFoot": 25,
    "goalLeftFoot": 26,
    "goalHead": 27,
    "goalObp": 28,
    "shortPassInaccurate": 29,
    "shortPassAccurate": 30,
    "passCorner": 31,
    "passCornerAccurate": 32,
    "passCornerInaccurate": 33,
    "passFreekick": 34,
    "passBack": 35,
    "passForward": 36,
    "passLeft": 37,
    "passRight": 38,
    "keyPassLong": 39,
    "keyPassShort": 40,
    "keyPassCross": 41,
    "keyPassCorner": 42,
    "keyPassThroughball": 43,
    "keyPassFreekick": 44,
    "keyPassThrowin": 45,
    "keyPassOther": 46,
    "assistCross": 47,
    "assistCorner": 48,
    "assistThroughball": 49,
    "assistFreekick": 50,
    "assistThrowin": 51,
    "assistOther": 52,
    "dribbleLost": 53,
    "dribbleWon": 54,
    "challengeLost": 55,
    "interceptionWon": 56,
    "clearanceHead": 57,
    "outfielderBlock": 58,
    "passCrossBlockedDefensive": 59,
    "outfielderBlockedPass": 60,
    "offsideGiven": 61,
    "offsideProvoked": 62,
    "foulGiven": 63,
    "foulCommitted": 64,
    "yellowCard": 65,
    "voidYellowCard": 66,
    "secondYellow": 67,
    "redCard": 68,
    "turnover": 69,
    "dispossessed": 70,
    "saveLowLeft": 71,
    "saveHighLeft": 72,
    "saveLowCentre": 73,
    "saveHighCentre": 74,
    "saveLowRight": 75,
    "saveHighRight": 76,
    "saveHands": 77,
    "saveFeet": 78,
    "saveObp": 79,
    "saveSixYardBox": 80,
    "savePenaltyArea": 81,
    "saveObox": 82,
    "keeperDivingSave": 83,
    "standingSave": 84,
    "closeMissHigh": 85,
    "closeMissHighLeft": 86,
    "closeMissHighRight": 87,
    "closeMissLeft": 88,
    "closeMissRight": 89,
    "shotOffTargetInsideBox": 90,
    "touches": 91,
    "assist": 92,
    "ballRecovery": 93,
    "clearanceEffective": 94,
    "clearanceTotal": 95,
    "clearanceOffTheLine": 96,
    "dribbleLastman": 97,
    "errorLeadsToGoal": 98,
    "errorLeadsToShot": 99,
    "intentionalAssist": 100,
    "interceptionAll": 101,
    "interceptionIntheBox": 102,
    "keeperClaimHighLost": 103,
    "keeperClaimHighWon": 104,
    "keeperClaimLost": 105,
    "keeperClaimWon": 106,
    "keeperOneToOneWon": 107,
    "parriedDanger": 108,
    "parriedSafe": 109,
    "collected": 110,
    "keeperPenaltySaved": 111,
    "keeperSaveInTheBox": 112,
    "keeperSaveTotal": 113,
    "keeperSmother": 114,
    "keeperSweeperLost": 115,
    "keeperMissed": 116,
    "passAccurate": 117,
    "passBackZoneInaccurate": 118,
    "passForwardZoneAccurate": 119,
    "passInaccurate": 120,
    "passAccuracy": 121,
    "cornerAwarded": 122,
    "passKey": 123,
    "passChipped": 124,
    "passCrossAccurate": 125,
    "passCrossInaccurate": 126,
    "passLongBallAccurate": 127,
    "passLongBallInaccurate": 128,
    "passThroughBallAccurate": 129,
    "passThroughBallInaccurate": 130,
    "passThroughBallInacurate": 131,
    "passFreekickAccurate": 132,
    "passFreekickInaccurate": 133,
    "penaltyConceded": 134,
    "penaltyMissed": 135,
    "penaltyWon": 136,
    "passRightFoot": 137,
    "passLeftFoot": 138,
    "passHead": 139,
    "sixYardBlock": 140,
    "tackleLastMan": 141,
    "tackleLost": 142,
    "tackleWon": 143,
    "cleanSheetGK": 144,
    "cleanSheetDL": 145,
    "cleanSheetDC": 146,
    "cleanSheetDR": 147,
    "cleanSheetDML": 148,
    "cleanSheetDMC": 149,
    "cleanSheetDMR": 150,
    "cleanSheetML": 151,
    "cleanSheetMC": 152,
    "cleanSheetMR": 153,
    "cleanSheetAML": 154,
    "cleanSheetAMC": 155,
    "cleanSheetAMR": 156,
    "cleanSheetFWL": 157,
    "cleanSheetFW": 158,
    "cleanSheetFWR": 159,
    "cleanSheetSub": 160,
    "goalConcededByTeamGK": 161,
    "goalConcededByTeamDL": 162,
    "goalConcededByTeamDC": 163,
    "goalConcededByTeamDR": 164,
    "goalConcededByTeamDML": 165,
    "goalConcededByTeamDMC": 166,
    "goalConcededByTeamDMR": 167,
    "goalConcededByTeamML": 168,
    "goalConcededByTeamMC": 169,
    "goalConcededByTeamMR": 170,
    "goalConcededByTeamAML": 171,
    "goalConcededByTeamAMC": 172,
    "goalConcededByTeamAMR": 173,
    "goalConcededByTeamFWL": 174,
    "goalConcededByTeamFW": 175,
    "goalConcededByTeamFWR": 176,
    "goalConcededByTeamSub": 177,
    "goalConcededOutsideBoxGoalkeeper": 178,
    "goalScoredByTeamGK": 179,
    "goalScoredByTeamDL": 180,
    "goalScoredByTeamDC": 181,
    "goalScoredByTeamDR": 182,
    "goalScoredByTeamDML": 183,
    "goalScoredByTeamDMC": 184,
    "goalScoredByTeamDMR": 185,
    "goalScoredByTeamML": 186,
    "goalScoredByTeamMC": 187,
    "goalScoredByTeamMR": 188,
    "goalScoredByTeamAML": 189,
    "goalScoredByTeamAMC": 190,
    "goalScoredByTeamAMR": 191,
    "goalScoredByTeamFWL": 192,
    "goalScoredByTeamFW": 193,
    "goalScoredByTeamFWR": 194,
    "goalScoredByTeamSub": 195,
    "aerialSuccess": 196,
    "duelAerialWon": 197,
    "duelAerialLost": 198,
    "offensiveDuel": 199,
    "defensiveDuel": 200,
    "bigChanceMissed": 201,
    "bigChanceScored": 202,
    "bigChanceCreated": 203,
    "overrun": 204,
    "successfulFinalThirdPasses": 205,
    "punches": 206,
    "penaltyShootoutScored": 207,
    "penaltyShootoutMissedOffTarget": 208,
    "penaltyShootoutSaved": 209,
    "penaltyShootoutSavedGK": 210,
    "penaltyShootoutConcededGK": 211,
    "throwIn": 212,
    "subOn": 213,
    "subOff": 214,
    "defensiveThird": 215,
    "midThird": 216,
    "finalThird": 217,
    "pos": 218
}

global_mapping = {val: key for key, val in original_mapping.items()}

In [9]:
from pyspark.sql import Row
import pyspark.sql.functions as sf

str_to_num_mapping_rows = [Row(event_type_value=camel_to_snake(v), event_type_id=k) for k, v in global_mapping.items()]
num_to_str_mapping_rows = [Row(event_type_id=k, event_type=camel_to_snake(v)) for k, v in global_mapping.items()]

global_mapping_df = spark.createDataFrame(num_to_str_mapping_rows)
global_mapping_df.show(10)

+-------------+------------------+
|event_type_id|        event_type|
+-------------+------------------+
|            0| shot_six_yard_box|
|            1| shot_penalty_area|
|            2|   shot_obox_total|
|            3|    shot_open_play|
|            4|      shot_counter|
|            5|    shot_set_piece|
|            6|shot_direct_corner|
|            7|   shot_off_target|
|            8|      shot_on_post|
|            9|    shot_on_target|
+-------------+------------------+
only showing top 10 rows



In [15]:
str_to_num_mapping_df = spark.createDataFrame(str_to_num_mapping_rows)
# str_to_num_mapping_df.show(10)

write_to_silver(str_to_num_mapping_df, "event_type_mapping")

Empty table created.


In [10]:
from pyspark.sql import functions as sf

# Lookup map for converting array of ids into value
lookup_map_df = global_mapping_df.groupBy().agg(
    sf.map_from_entries(
        sf.collect_list(sf.struct("event_type_id", "event_type"))
    ).alias("map")
)

fct_match_events_df = (
    base_match_events_df.crossJoin(lookup_map_df)
    .withColumn(
        "satisfied_events_types_names",
        sf.transform(
            "satisfied_events_types",
            lambda x: sf.element_at(sf.col("map"), x)
        )
    )
    .drop("map")
)


In [11]:
fct_match_events_columns = [
    # Keys 
    "match_id",
    "_event_id",
    "event_id",
    "team_id",
    "player_id",

    # Time and Period
    "minute",
    "second",
    "period_value",
    "period_display_name",

    # Event Details
    "type_display_name",
    "is_successful",

    # Location
    "x",
    "y",
    "end_x",
    "end_y",

    # Enriched Qualifier Data
    "qualifiers_display_names",
    "satisfied_events_types_names",
    "qualifiers_values",
    "satisfied_events_types",
    
    # Partitioning Columns
    "league",
    "season",
    "stage_id"
]

# --- 2. Apply the final projection ---
fct_match_events_df = fct_match_events_df.select(fct_match_events_columns)

In [25]:
fct_match_events_df.show(truncate=False)

25/08/21 17:47:16 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+---------+--------+-------+---------+------+------+------------+-------------------+-----------------+-------------+----+----+-----+-----+------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+----------------------------------------------+----------------------------+------+--------+
|match_id|_event_id|event_id|team_id|player_id|minute|second|period_value|period_display_name|type_display_name|is_successful|x   |y   |end_x|end_y|qualifiers_display_names|satisfied_events_types_names                                                                                                                                     |qualifiers_values|satisfied_events_types                        |league                      |season|stage_id|
+--------+---------+--------+-------+---------+------+------+------------+-------------------+--------------

In [22]:
import pyspark.sql.functions as sf

joined_match_events_df = (
    exploded_match_events_df
    .join(
        sf.broadcast(global_mapping_df),
        on=["event_type_id"],
        how="left"
    )
)

In [23]:
from pyspark.sql.functions import collect_list

qualifiers_lookup_df = (
    joined_match_events_df
    .groupBy(["surrogate_event_id"])
    .agg(
        collect_list("event_type").alias("event_types_list")
    )
)

In [24]:
fct_match_events_df = base_match_events_df.join(
    qualifiers_lookup_df,
    on="surrogate_event_id",
    how="left"
).drop(col("surrogate_event_id"))

In [25]:
# print(fct_match_events_df.count())
fct_match_events_df.printSchema()
fct_match_events_df.show()

root
 |-- match_id: integer (nullable = true)
 |-- _event_id: integer (nullable = false)
 |-- event_id: integer (nullable = true)
 |-- minute: integer (nullable = true)
 |-- second: integer (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- player_id: integer (nullable = true)
 |-- x: float (nullable = true)
 |-- y: float (nullable = true)
 |-- end_x: float (nullable = true)
 |-- end_y: float (nullable = true)
 |-- period_value: integer (nullable = true)
 |-- period_display_name: string (nullable = true)
 |-- type_value: integer (nullable = true)
 |-- type_display_name: string (nullable = true)
 |-- outcome_type_value: integer (nullable = true)
 |-- outcome_type_display_name: string (nullable = true)
 |-- satisfied_events_types: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- qualifiers_list: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- league: string (nullable = true)
 |-- season: string (nullable = true)
 |-- s

+--------+---------+--------+------+------+-------+---------+----+----+-----+-----+------------+-------------------+----------+-----------------+------------------+-------------------------+----------------------+--------------------+--------------------+---------+--------+--------------------+
|match_id|_event_id|event_id|minute|second|team_id|player_id|   x|   y|end_x|end_y|period_value|period_display_name|type_value|type_display_name|outcome_type_value|outcome_type_display_name|satisfied_events_types|     qualifiers_list|              league|   season|stage_id|    event_types_list|
+--------+---------+--------+------+------+-------+---------+----+----+-----+-----+------------+-------------------+----------+-----------------+------------------+-------------------------+----------------------+--------------------+--------------------+---------+--------+--------------------+
|  789618|       12|       8|     0|    39|    338|     9909|24.6|83.2|  8.4| 58.1|           1|          FirstH

In [12]:
write_to_silver(fct_match_events_df, "fct_match_events", is_league_partition=True)

Empty table created.


25/08/21 17:55:30 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [ ]:
from pyspark.sql.functions import max as spark_max, min as spark_min

fct_match_events_df.filter(
    col("match_id") == 1834247
).orderBy(
    col("_event_id").asc(),
).show(n=20)

+--------+---------+--------+------+------+-------+---------+----+-----+------------+-------------------+----------+-----------------+------------------+-------------------------+----------------------+------------------+---------+--------+--------------------+
|match_id|_event_id|event_id|minute|second|team_id|player_id|   x|    y|period_value|period_display_name|type_value|type_display_name|outcome_type_value|outcome_type_display_name|satisfied_events_types|            league|   season|stage_id|     qualifiers_list|
+--------+---------+--------+------+------+-------+---------+----+-----+------------+-------------------+----------+-----------------+------------------+-------------------------+----------------------+------------------+---------+--------+--------------------+
| 1834247|        1|       2|     0|     0|     36|     NULL| 0.0|  0.0|           1|          FirstHalf|        32|            Start|                 1|               Successful|                    []|germany-bund

Check if match event mapping matches global_mapping

In [ ]:
# think this is not really needed but can't hurt to check just in case
from pyspark.sql.types import StructType, StructField, MapType, StringType, IntegerType, MapType
from pyspark.sql.functions import broadcast

mapping_path = "/home/dottier/big_data/bronze/match_data"

mapping_schema = StructType([
    StructField("matchCentreEventTypeJson", MapType(StringType(), IntegerType()), True)
])

mapping_df = spark.read \
    .schema(mapping_schema) \
    .option("multiLine", True) \
    .option("pathGlobFilter", "*match_data.json") \
    .json(mapping_path)


exploded_mapping_df = mapping_df.select(
    "match_id", "league", "season", "stage_id",
    explode("matchCentreEventTypeJson").alias("event_type", "event_type_id")
)
exploded_mapping_df.printSchema()

joined = exploded_mapping_df.join(
    broadcast(global_mapping_df),
    exploded_mapping_df["event_type"] == global_mapping_df["event_type"],
    how="left"
).select(
    exploded_mapping_df["*"],
    global_mapping_df["event_type_id"].alias("global_value")
)

mismatches = joined.filter(
    (col("event_type_id") != col("global_value")) | col("global_value").isNull()
)

if mismatches.limit(1).count() > 0:
    mismatches.show(truncate=False)
    raise ValueError("❌ Found mismatches in matchCentreEventTypeJson")
else:
    print("✅ All matchCentreEventTypeJson match the global mapping.")


root
 |-- match_id: integer (nullable = true)
 |-- league: string (nullable = true)
 |-- season: string (nullable = true)
 |-- stage_id: integer (nullable = true)
 |-- event_type: string (nullable = false)
 |-- event_type_id: integer (nullable = true)



✅ All matchCentreEventTypeJson match the global mapping.


In [ ]:
from pyspark.sql import functions as sf

# make it a list of tuple
df = spark.createDataFrame([("4 : 1",), ("2 : 1",), ("5 : 0",)], ["score"])
df.show()

df.select(sf.regexp_extract(sf.split(sf.col("score"), ":").getItem(0))).show()

+-----+
|score|
+-----+
|4 : 1|
|2 : 1|
|5 : 0|
+-----+

+----------------------------+
|trim(split(score, :, -1)[0])|
+----------------------------+
|                           4|
|                           2|
|                           5|
+----------------------------+



In [ ]:
from pyspark.sql import functions as sf

# make it a list of tuple
df = spark.createDataFrame([(1, 2, 3, 4), (5, 6, 7, 8)], ["val1", "val2", "val3", "val4"])
df.show()


# flattened_df = df.select(sf.explode(sf.array("val1", "val2")).alias("value"), sf.col("val3"))
# flattened_df.show()

+----+----+----+----+
|val1|val2|val3|val4|
+----+----+----+----+
|   1|   2|   3|   4|
|   5|   6|   7|   8|
+----+----+----+----+

+-----+----+
|value|val3|
+-----+----+
|    1|   3|
|    2|   3|
|    5|   7|
|    6|   7|
+-----+----+



In [17]:
from pyspark.sql import functions as sf

# make it a list of tuple
df = spark.createDataFrame([([1, 2, 3],), ([4, 5, 6],)], ["val1"])
df.show()


# flattened_df = df.select(sf.explode(sf.array("val1", "val2")).alias("value"), sf.col("val3"))
# flattened_df.show()

+---------+
|     val1|
+---------+
|[1, 2, 3]|
|[4, 5, 6]|
+---------+



In [16]:
map_df = spark.createDataFrame([(1, 'a'), (2, 'b'), (3, 'c'),
                                (4, 'd'), (5, 'e'), (6, 'f')], ["key", "val"])
map_df.show()

+---+---+
|key|val|
+---+---+
|  1|  a|
|  2|  b|
|  3|  c|
|  4|  d|
|  5|  e|
|  6|  f|
+---+---+



In [19]:
from pyspark.sql import functions as sf

# map_df: (key:int, val:string)
# df: val1: array<int>

# 1) Aggregate map_df into a single-row map column
lookup_map_df = map_df.groupBy().agg(
    sf.map_from_entries(sf.collect_list(sf.struct("key", "val"))).alias("m")
)

lookup_map_df.show()

# 2) Cross join once to make the map available, then transform the array
result_df = (
    df.crossJoin(sf.broadcast(lookup_map_df))
      .withColumn("val1", sf.transform("val1", lambda x: sf.element_at(sf.col("m"), x)))
      .select(df['*'])
)

result_df.show(truncate=False)


+--------------------+
|                   m|
+--------------------+
|{1 -> a, 2 -> b, ...|
+--------------------+



AnalysisException: [MISSING_ATTRIBUTES.RESOLVED_ATTRIBUTE_APPEAR_IN_OPERATION] Resolved attribute(s) "val1" missing from "val1", "m" in operator !Project [val1#2157]. Attribute(s) with the same name appear in the operation: "val1".
Please check if the right attribute(s) are used.;
!Project [val1#2157]
+- Project [transform(val1#2157, lambdafunction(element_at(m#2333, lambda x_2#2415L, None, false), lambda x_2#2415L, false)) AS val1#2414, m#2333]
   +- Join Cross
      :- LogicalRDD [val1#2157], false
      +- ResolvedHint (strategy=broadcast)
         +- Aggregate [map_from_entries(collect_list(struct(key, key#2144L, val, val#2145), 0, 0)) AS m#2333]
            +- LogicalRDD [key#2144L, val#2145], false


In [27]:
def mapping(key):
    return global_mapping.get(key, -1)

print(mapping(1))

shotPenaltyArea


In [29]:
from pyspark.sql.functions import transform, lit
df.select(transform("val1", lambda x: lit(global_mapping[x]))).show()

TypeError: unhashable type: 'Column'

In [7]:
import json

base_path = "/home/dottier/big_data/bronze/season_data/league=germany_bundesliga/season=2023_2024/season_info.json"

with open(base_path, "r", encoding="utf-8") as f:
    json_object = json.load(f)
    print(json_object)

{'tournamentId': 3, 'stageId': 22128, 'stageName': 'Bundesliga', 'regionId': 81, 'tournamentName': 'Bundesliga', 'seasonName': '2023/2024', 'seasonId': 9649, 'stageSortOrder': 1, 'sex': 1, 'tournamentSortOrder': 170, 'regionCode': 'de', 'regionName': 'Germany', 'isOpta': True, 'navigationDisplayMode': 2, 'matches': [{'stageId': 22128, 'id': 1743688, 'status': 6, 'startTime': '2024-05-18T14:30:00', 'homeTeamId': 44, 'homeTeamName': 'Borussia Dortmund', 'homeYellowCards': 0, 'homeRedCards': 0, 'awayTeamId': 1147, 'awayTeamName': 'Darmstadt', 'awayYellowCards': 3, 'awayRedCards': 0, 'hasIncidentsSummary': True, 'hasPreview': True, 'scoreChangedAt': '2024-05-18 16:17:02Z', 'elapsed': 'FT', 'lastScorer': 0, 'isTopMatch': False, 'homeTeamCountryCode': 'de', 'awayTeamCountryCode': 'de', 'commentCount': 4, 'isLineupConfirmed': True, 'isStreamAvailable': False, 'matchIsOpta': False, 'homeTeamCountryName': 'Germany', 'awayTeamCountryName': 'Germany', 'startTimeUtc': '2024-05-18T13:30:00Z', 'home

In [32]:
from pyspark.sql import functions as F

mapping = {0: "a", 1: "b", 2: "c"}

# Your DataFrame has a column "nums" like: [0, 1, 2]
df = spark.createDataFrame(
    [([0, 1, 2],), ([1, 0],)],
    ["nums"]
)

# Transform each element in the array
df = df.withColumn(
    "letters",
    F.expr(f"transform(nums, x -> '{mapping[0]}' if x = 0 else '{mapping[1]}' if x = 1 else '{mapping[2]}')")
)

df.show(truncate=False)


ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'if': extra input 'if'.(line 1, pos 25)

== SQL ==
transform(nums, x -> 'a' if x = 0 else 'b' if x = 1 else 'c')
-------------------------^^^


In [ ]:
from pyspark.sql import Row
import pyspark.sql.functions as sf

mapping_rows = [Row(event_type_id=k, event_type=v) for k, v in global_mapping.items()]

global_mapping_df = spark.createDataFrame(mapping_rows)
global_mapping_df = global_mapping_df.groupBy().agg(
    sf.map_from_entries(sf.collect_list(sf.struct("event_type_id", "event_type"))).alias("map")
)

fct_match_events_df = (
    fct_match_events_df
    .crossJoin(sf.broadcast(global_mapping_df))
    .withColumn("satisfied_events_types", sf.transform("satisfied_events_types", lambda x: sf.element_at(sf.col("map"), x)))
    .drop("map")
)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `satisfied_events_types` cannot be resolved. Did you mean one of the following? [`_event_id`, `event`, `match_id`, `season`, `stage_id`].;
'Project [league#1881, season#1882, stage_id#1883, match_id#1884, event#1893, _event_id#1906, map#2033, transform('satisfied_events_types, lambdafunction(element_at('map, lambda 'x_0, None, false), lambda 'x_0, false)) AS satisfied_events_types#2042]
+- Join Cross
   :- Project [league#1881, season#1882, stage_id#1883, match_id#1884, event#1893, _event_id#1906]
   :  +- Project [league#1881, season#1882, stage_id#1883, match_id#1884, event#1893, _w0#1907L, _event_id#1906, _event_id#1906]
   :     +- Window [row_number() windowspecdefinition(match_id#1884, _w0#1907L ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS _event_id#1906], [match_id#1884], [_w0#1907L ASC NULLS FIRST]
   :        +- Project [league#1881, seas

In [4]:
fct_match_events = read_from_silver("fct_match_events")

In [ ]:


events_for_join = fct_match_events_df.select(
    "match_id",
    "_event_id",
    "minute",
    "second",
    "type_display_name",
    "outcome_type_display_name",
    "player_id",
    "team_id",
    "qualifiers_list"
)

event1 = events_for_join.alias("event1")
event2 = events_for_join.alias("event2")

join_conditions = [
    col("event1.match_id") == col("event2.match_id"), # Must be in the same match
    col("event1.minute") == col("event2.minute"),
    col("event1.second") == col("event2.second"),
    col("event1.team_id") < col("event2.team_id"),
    col("event1._event_id") < col("event2._event_id")
]

event_pairs_df = event1.join(event2, on=join_conditions, how="inner")

print(event_pairs_df.count())
event_pairs_df.show()

787004


+--------+---------+------+------+-----------------+-------------------------+---------+-------+--------------------+--------+---------+------+------+-----------------+-------------------------+---------+-------+--------------------+
|match_id|_event_id|minute|second|type_display_name|outcome_type_display_name|player_id|team_id|     qualifiers_list|match_id|_event_id|minute|second|type_display_name|outcome_type_display_name|player_id|team_id|     qualifiers_list|
+--------+---------+------+------+-----------------+-------------------------+---------+-------+--------------------+--------+---------+------+------+-----------------+-------------------------+---------+-------+--------------------+
|  789612|      172|     8|    51|           Aerial|               Successful|     9909|    338|[duelAerialWon, o...|  789612|      173|     8|    51|           Aerial|             Unsuccessful|    25244|    418|[duelAerialLost, ...|
|  789612|      967|    57|    27|      BlockedPass|            

In [12]:
from pyspark.sql.functions import lag, col
from pyspark.sql.window import Window

# Define a window that partitions by match and orders by our _event_id
# This puts the events in the sequence we want to test.
window_spec = Window.partitionBy("match_id").orderBy("_event_id")

# Create columns for the minute and second of the *previous* event
events_with_previous_time = fct_match_events.withColumn(
    "previous_minute", lag("minute").over(window_spec)
).withColumn(
    "previous_second", lag("second").over(window_spec)
).withColumn(
    "previous_period", lag("period_value").over(window_spec)
)

# Now, find the rows where the time goes "backwards".
# This is our alarm for incorrect ordering.
ordering_errors = events_with_previous_time.filter(
    (col("minute") < col("previous_minute")) | 
    ((col("minute") == col("previous_minute")) & (col("second") < col("previous_second")))
).filter(
    col("previous_period") == col("period_value")
)

# The assertion is a simple count.
error_count = ordering_errors.count()

if error_count > 0:
    print(f"DATA QUALITY FAILED: Found {error_count} events that are out of chronological order!")
    # Show the specific events that broke the rule
    ordering_errors.select(
        "match_id", "_event_id", "minute", "second", "previous_minute", "previous_second"
    ).show()
else:
    print("DATA QUALITY PASSED: Event order is chronologically correct.")

DATA QUALITY PASSED: Event order is chronologically correct.


In [19]:
from pyspark.sql.functions import array_contains, col

fct_match_events.filter(
    col("match_id") == 959673
    # (col("_event_id") == 1) &
    # (col("type_display_name") != "Start") &
    # (col("second").isNotNull())
    # (array_contains(col("qualifiers_list"), "redCard"))
).orderBy(
    "_event_id"
).show(n=10000)

+--------+---------+--------+------+------+-------+---------+----+-----+------------+-------------------+----------+-----------------+------------------+-------------------------+----------------------+--------------------+---------+--------+--------------------+
|match_id|_event_id|event_id|minute|second|team_id|player_id|   x|    y|period_value|period_display_name|type_value|type_display_name|outcome_type_value|outcome_type_display_name|satisfied_events_types|              league|   season|stage_id|     qualifiers_list|
+--------+---------+--------+------+------+-------+---------+----+-----+------------+-------------------+----------+-----------------+------------------+-------------------------+----------------------+--------------------+---------+--------+--------------------+
|  959673|        1|       3|     0|     0|     96|    12480|50.1| 50.1|           1|          FirstHalf|         1|             Pass|                 1|               Successful|  [91, 119, 117, 30...|englan